# Case studies for Robot Test paper

In [ ]:
import os
import pandas as pd
import numpy as np
import datetime

import run_robot
import prepare_data
from importlib import reload
reload(prepare_data)
reload(run_robot)

In [ ]:
basic_prm = prepare_data.save_basic_parameters(min_level=0.8, rep=2.5, ndays=30)
subnot_factor = 11.6
cities_data = prepare_data.compute_initial_condition_evolve_and_save(basic_prm, "SP", ["Mun. São Paulo"], 10000000, subnot_factor, 1, "data/covid_with_drs_07_29.csv")
cities_data

In [ ]:
# Define the basic data for the case studies

# Basic reproduction number
basic_rep = 1.8

# Simulation horizon
# A little more than a year when thevaccine should be here
ndays = 14*2*14

# Time series
need_icu = 0.00693521103887298
time_icu = 7
# need_icu = 0.00650997
# time_icu = 11

# Lockdown level
lock_level = 0.8

# Define basic paramters
basic_prm = prepare_data.save_basic_parameters(min_level=lock_level, rep=basic_rep, ndays=ndays, need_icu=need_icu, time_icu=time_icu)

# Compute initial values

# For cities
# cities_data = prepare_data.compute_initial_condition_evolve_and_save(basic_prm, "SP", ["Araçatuba", "São José Do Rio Preto"], 500000, 1)
# cities_data = prepare_data.compute_initial_condition_evolve_and_save(basic_prm, "SP", ["São José Do Rio Preto"], 25000, 6, 1)

# For DRS
cities_data = prepare_data.compute_initial_condition_evolve_and_save(basic_prm, "SP", [], 000000, subnot_factor, 1, "data/covid_with_drs_07_01.csv")

# Sub-groups for figures
sp = ["Mun. São Paulo"]
sp_so = sp + ["Sub região sudoeste - RMSP"]
rmsp = sp + ["Sub região leste - RMSP", "Sub região norte - RMSP", "Sub região oeste - RMSP", "Sub região sudeste - RMSP", "Sub região sudoeste - RMSP"]

In [ ]:
# Create a target matrix (max infected level)
ncities, ndays = len(cities_data.index), int(basic_prm["ndays"])
target = 0.8*np.ones((ncities, ndays))
target = prepare_data.save_target(cities_data, target)

# Use a forcedif that releases the cities in the end
force_dif = np.ones((ncities, ndays))
cities_data

In [ ]:
pd.set_option("display.width", 120)

# Simple function to run a test and save results
def run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif, pools=None, budget=0, verbosity=1):
    hammer_data = prepare_data.save_hammer_data(cities_data, 0, basic_prm["min_level"])
    run_robot.find_feasible_hammer(basic_prm, cities_data, M, target, hammer_data, out_file=None, 
        incr_all=True, verbosity=verbosity)
    run_robot.prepare_optimization(basic_prm, cities_data, M, target, hammer_data, force_dif, pools, 
        verbosity=verbosity, test_budget=budget)
    run_robot.optimize_and_show_results(basic_prm, figure_file, result_file, cities_data, verbosity=verbosity)
    

In [ ]:
# Define mobility matrix.
M = prepare_data.convert_mobility_matrix_and_save(cities_data, max_neighbors=5, drs=True)
M.loc["Mun. São Paulo", "Sub região sudoeste - RMSP"], M.loc["Sub região sudoeste - RMSP", "Mun. São Paulo"]


## Case 1: 14 day window, official data

In [ ]:
basic_prm["alternate"] = 0.0
budget = 20000000
result_file =  f"results/tests_new_{budget:d}_daily_50k_with_mobility_sp.csv"
figure_file = f"results/tests_new_{budget:d}_daily_50k_with_mobility_sp.png"
run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif, budget=budget)